In [ ]:
from pydantic import BaseModel,Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser
import os
import json
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
import re
from typing import List, Dict, Any, Optional
from enum import Enum
from langchain.output_parsers import PydanticOutputParser




class SectionType(str, Enum):
    CONTENT = "content"
    INFO = "info"
    CODE = "code"
    TIP = "tip"

class Section(BaseModel):
    """Model for a lesson section"""
    type: SectionType = Field(description="Type of section")
    title: str = Field(description="Section title")
    content: str = Field(description="Main content")
    language: Optional[str] = Field(default=None, description="Programming language for code sections")
    explanation: Optional[str] = Field(default=None, description="Explanation for code sections")

class DetailedChapter(BaseModel):
    """Model for a detailed chapter with rich content"""
    id: int = Field(description="Chapter ID")
    title: str = Field(description="Chapter title")
    duration: str = Field(description="Duration in readable format (e.g., '15 min')")
    type: str = Field(default="lesson", description="Chapter type")
    sections: List[Section] = Field(description="List of chapter sections")


class Chapter(BaseModel):
    """Model for a course chapter"""
    chapter_number: int = Field(description="Chapter number")
    title: str = Field(description="Chapter title")
    description: str = Field(description="Brief description of chapter content")
    learning_objectives: List[str] = Field(description="Key learning objectives")
    estimated_duration: int = Field(description="Estimated time to complete in Minutes")

class CourseOutline(BaseModel):
    """Model for complete course outline"""
    course_title: str = Field(description="Title of the course")
    course_description: str = Field(description="Brief course description")
    level: str = Field(description="Level of course")
    total_chapters: int = Field(description="Total number of chapters")
    duration: int   = Field(description="Time required to complete the course in Minutes")
    chapters: List[Chapter] = Field(description="List of chapters")
        


os.environ["GROQ_API_KEY"] = GROQ_API_KEY

llm = ChatGroq(
            temperature=0.3,
            model_name="openai/gpt-oss-20b",
            max_tokens=4000
        )
# llm = ChatGoogleGenerativeAI(
#     model="gemini-pro",
#     google_api_key=os.getenv("GOOGLE_API_KEY"),
# )


In [66]:
def generate_course(name,target_audiunce,difficulty,duration):
    
    llm_chain = llm.with_structured_output(CourseOutline)
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are an expert course designer and educator. 
    Your task is to create a complete, well-structured course outline.

    Guidelines:
    - The course should have a logical progression.
    - Each chapter should build on the previous one.
    - Learning objectives must be actionable and clear.
    - Estimated durations should align with the total course duration.
    - Keep explanations concise, student-friendly, and practical."""),

        ("human", """Create a course outline for the topic: {course_topic}

    Requirements:
    - Target audience: {target_audience}
    - Course difficulty: {difficulty_level}
    - Total course duration: {course_duration} Months""")
    ])

    chain = prompt_template | llm_chain
    result = chain.invoke({
        "course_topic": name,
        "target_audience":target_audiunce,
        "difficulty_level": difficulty,
        "course_duration": duration
    })

    return result



In [72]:
def generate_chapter_content(chapters):
    idx = 0
    parser = PydanticOutputParser(pydantic_object=DetailedChapter)

    for chapter in chapters:
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", """You are an expert programming instructor. 
        Your task is to expand a course chapter into detailed lesson sections.

        Guidelines:
        - Divide the chapter into multiple sections of type: content, info, code, or tip.
        - CONTENT → main explanation of a concept, written simply for students.
        - INFO → short fact, rule, or important clarification.
        - CODE → code examples (must include "language" and "explanation").
        - TIP → best practices, pitfalls, or learning hacks.
        - Keep sections concise but informative.
        - The final output MUST be valid JSON that matches the following schema:
        {format_instructions}
        """),

            ("human", """Expand the following chapter into structured sections:

        Chapter {chapter_number}: {chapter_title}
        Description: {chapter_description}
        Learning Objectives: {learning_objectives}
        Estimated Duration: {estimated_duration} minutes
        """)
        ]).partial(format_instructions=parser.get_format_instructions())

        
        llm_chain = llm.with_structured_output(DetailedChapter)
        
        chain = prompt_template | llm | parser

        result = chain.invoke({
            "chapter_number": chapter.chapter_number,
            "chapter_title": chapter.title,
            "chapter_description": chapter.description,
            "learning_objectives": chapter.learning_objectives,
            "estimated_duration": chapter.estimated_duration
        })

        idx += 1
        if idx == 5:
            break
    return result


In [68]:
course = generate_course(name="Python",difficulty="Easy",target_audiunce="Begineer",duration=2)

In [73]:
chapters = generate_chapter_content(course.chapters)

In [74]:
chapters.dict()

{'id': 5,
 'title': 'Functions and Modules',
 'duration': '120 min',
 'type': 'lesson',
 'sections': [{'type': <SectionType.CONTENT: 'content'>,
   'title': 'What is a Function?',
   'content': 'A function is a reusable block of code that performs a specific task. It can accept input values called arguments and may return a result. Functions help keep code organized, readable, and maintainable.',
   'language': None,
   'explanation': None},
  {'type': <SectionType.INFO: 'info'>,
   'title': 'Function Definition Syntax',
   'content': '```python\ndef function_name(arg1, arg2):\n    # body\n    return result\n```',
   'language': None,
   'explanation': None},
  {'type': <SectionType.CODE: 'code'>,
   'title': 'Basic Function with Positional Arguments',
   'content': 'def greet(name, message):\n    print(f"{message}, {name}!")\n\ngreet("Alice", "Hello")\n',
   'language': 'python',
   'explanation': 'The function greet takes two positional arguments and prints a greeting. Arguments are 